# Importando as bibliotecas necessárias

In [49]:
from binance import Client
import pandas as pd
import time
import datetime
import matplotlib.pyplot as plt
import mplfinance as mpf

# Setando a autenticação da API

In [50]:
#Coloque seu token de autenticação aqui

API_Key= ''
    
API_Secret= ''

client = Client(API_Key,API_Secret)

# Setando as datas de início e fim do estudo

In [51]:
#timestamp das datas 11/06/2024 a 26/06/2024:

start_time = 1718064000000

end_time = 1719360000000

# Obtendo os dados de preço do BTC no intervalo desejado

In [52]:
#envio de requisição ao client da Binance dos dados do mercado futuro para o ativo BTC/USDT no intervalo gráfico de 5min para o período desejado 
klines_five = client.futures_historical_klines("BTCUSDT", "5m",start_str=start_time,end_str=end_time)

#transformação dos dados em formato json para dataframe
df_price = pd.DataFrame(klines_five)

#remoção de colunas não desejadas
df_price = df_price.drop([6,7,8,9,10,11],axis=1)

#renomear os números de colunas
df_price = df_price.rename(columns={0:'Open Time',1:'Open',2:'High',3:'Low',4:'Close',5:'Volume'})

#transformação dos tipos de dados
df_price[['Open','High','Low','Close','Volume']] = df_price[['Open','High','Low','Close','Volume']].astype(float)

#transformação do dado de data/hora
df_price['Open Time'] =  df_price['Open Time'].astype('datetime64[ms]')

#Salvando os dados em um arquivo csv
df_price.to_excel(r'C:\Users\rober\Documents\price_data.xlsx',index=False)

# Obtendo os dados do Indicador Long/Short Ratio

In [53]:
# Definindo os parâmetros
symbol = "BTCUSDT"
interval = '5m'

#Limite de registros por requisição (máx: 500)
limit = 500 

#Limite de requisições por minuto
requests_per_minute = 1200  

#Tempo de espera entre as requisições
sleep_time = 60 / requests_per_minute  

#Função para obter o long/short ratio
def get_longshort_ratio(client, symbol, start_time, end_time, interval, limit=500):
    ratio = client.futures_global_longshort_ratio(symbol=symbol, period=interval, startTime=start_time, endTime=end_time, limit=limit)
    return ratio

#Lista para armazenar todos os dados
all_ratios = []

#Início da iteração com start_time e end_time ajustados
current_start_time = start_time

while current_start_time < end_time:
    #Definir o próximo end_time baseado no intervalo e limite
    current_end_time = min(current_start_time + limit * 300000, end_time)  # 300000 ms = 5 minutos

    #Obter os dados do long/short ratio
    ratios = get_longshort_ratio(client, symbol, current_start_time, current_end_time, interval, limit)

    if not ratios:
        break

    #Adicionar dados ao conjunto de todos os dados
    all_ratios.extend(ratios)
    
    #Atualizar o start_time para a próxima iteração
    current_start_time = current_end_time

    #Aguardar antes de fazer a próxima requisição para respeitar a limitação
    time.sleep(sleep_time)

#Convertendo os dados para DataFrame
ratios_df = pd.DataFrame(all_ratios)

#Renomeando as colunas, se necessário
ratios_df = ratios_df.rename(columns={
    'longShortRatio': 'Long/Short Ratio',
    'longAccount': 'Long Account',
    'shortAccount': 'Short Account',
    'timestamp': 'Timestamp'
})
# Ajustar os tipos de dados do DataFrame final
ratios_df['Timestamp'] = pd.to_datetime(ratios_df['Timestamp'], unit='ms')

#renomeia a coluna
ratios_df = ratios_df.rename(columns={'Timestamp':'Datetime'})
                             
#Salvando os dados em um arquivo csv
ratios_df.to_excel(r'C:\Users\rober\Documents\longshort_ratio.xlsx',index=False)


# Obtendo os dados do Open Interest

In [54]:
#Função para converter data para timestamp
def date_to_timestamp(date_str):
    return int(pd.Timestamp(date_str).timestamp() * 1000)

#Parâmetros
symbol = "BTCUSDT"
interval = "5m"
start_date = "2024-06-11"
end_date = "2024-06-26"
limit = 500
one_day = 1440 * 60 * 1000  #1440 minutos em milissegundos

#Convertendo datas para timestamps
start_timestamp = date_to_timestamp(start_date)
end_timestamp = date_to_timestamp(end_date)

#DataFrame para armazenar os dados
all_data = pd.DataFrame()

#Iterando pelas requisições para obter todos os dados no intervalo desejado
current_start_timestamp = start_timestamp
while current_start_timestamp < end_timestamp:
    current_end_timestamp = min(current_start_timestamp + one_day, end_timestamp)

    #Obter os dados da API
    data = client.futures_open_interest_hist(
        symbol=symbol,
        period=interval,
        startTime=current_start_timestamp,
        endTime=current_end_timestamp,
        limit=limit
    )
    
    if not data:
        break

    #Converter para DataFrame
    df = pd.DataFrame(data)

    #Concatenar com o DataFrame total
    all_data = pd.concat([all_data, df], ignore_index=True)

    #Atualizar o current_start_timestamp para a próxima iteração
    current_start_timestamp = current_end_timestamp + 1

    #Pausa para respeitar limites de requisições
    time.sleep(1)

#Ajustar os tipos de dados do DataFrame final
all_data['timestamp'] = pd.to_datetime(all_data['timestamp'], unit='ms')

#Renomeia a coluna timestamp para o novo formato datetime
all_data = all_data.rename(columns={'timestamp':'Datetime'})

#Remove os valores duplicados
all_data = all_data.drop_duplicates()

#Exportar o DataFrame para um arquivo csv
all_data.to_excel(r'C:\Users\rober\Documents\open_interest.xlsx',index=False)